In [ ]:
import pandas as pd
import random
import string
#from tqdm import tqdm_notebook as tqdm
from future.builtins import next

import os
import sys
import csv
import re
import collections
import logging
import optparse
from numpy import nan
from tqdm import tqdm

import dedupe

from unidecode import unidecode

maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [ ]:
df=pd.read_csv("Cadwell_List_2.txt",sep='\t',header=None,dtype=str,names=['First_Name','Middle_Name','Last_Name','Address1','Address2','City','State','Zip','Lat','Long','Phonenumber','Sex','Age'],low_memory=False)

In [ ]:

da_f=df.loc[:,[
            'First_Name',
            'Middle_Name',
            'Last_Name','Address1','Address2','City','Zip','Phonenumber']]
#da_f.head()
da_f[da_f['Zip']=='84128'].to_csv("Caldwell_List_3.csv",header=True,index=True,index_label='id',encoding='utf8')
da_f.to_csv("Caldwell_List_2.csv",header=True,index=True,index_label='id',encoding='utf8')

In [ ]:
dm=pd.read_csv('Data_Model',dtype=str)
dm_r=dm.rename(columns={'statevoterid':'id',
'vf_firstname':'First_Name',
'MiddleInitial':'Middle_Name',
'vf_lastname':'Last_Name',
'RegistrationAddress1':'Address1',
'RegistrationAddress2':'Address2',
'RegistrationAddressCity':'City',
'RegistrationAddressZip5':'Zip',
'landline':'Phonenumber'})



In [ ]:
dm_f=dm_r.loc[:,['id',
            'First_Name',
            'Middle_Name',
            'Last_Name','Address1','Address2','City','Zip','Phonenumber']]
dm_f.to_csv("Data_Model_dedupe.csv",header=True,index=False,encoding='utf8')

In [ ]:
#da=pd.read_csv("Caldwell_List.csv")
dm=pd.read_csv('Data_Model_dedupe.csv',dtype=str)
dm[dm['Zip']=='84128'].to_csv('Data_Model_dedupe_1.csv',header=True,index=False,encoding='utf8')


In [ ]:
dm1=pd.read_csv('Data_Model_dedupe_1.csv',dtype=str)
dm1

In [ ]:
input_filet = 'Caldwell_List_3.csv'
input_filet1= 'Data_Model_dedupe_1.csv'
input_file = 'Caldwell_List_2.csv'
input_file1= 'Data_Model_dedupe.csv'
output_file = 'data_matching_output.csv'
settings_file = 'data_matching_learned_settings'
training_file = 'data_matching_training.json'


def preProcess(column):
#
    column = unidecode(column)
    column = re.sub('\n', ' ', column)
    column = re.sub('-', '', column)
    column = re.sub('/', ' ', column)
    column = re.sub("'", '', column)
    column = re.sub(",", '', column)
    column = re.sub(":", ' ', column)
    column = re.sub('  +', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column :
        column = None
    return column

def readData(filename):
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """

    data_d = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in tqdm(reader):
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['id'])
            data_d[row_id] = dict(clean_row)

    return data_d


print('importing data ...')
data_1 = readData(input_file)
data_2 = readData(input_file1)
data_3 = readData(input_filet)
data_4 = readData(input_filet1)

In [ ]:
data_1

In [ ]:
# ## Training

if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)

else:
    # Define the fields dedupe will pay attention to
    #
    # Notice how we are telling dedupe to use a custom field comparator
    # for the 'Zip' field.
    fields = [
        {'field' : 'First_Name', 'type': 'String'},
        {'field' : 'Middle_Name', 'type': 'String'},
        {'field' : 'Last_Name', 'type': 'String'},
        {'field' : 'Address1', 'type': 'String'},
        {'field' : 'Address2', 'type': 'String','has missing' : True},
        {'field' : 'City', 'type': 'String'},
        {'field' : 'Zip', 'type': 'Exact', 'has missing' : False},
        {'field' : 'Phonenumber', 'type': 'String', 'has missing' : True},
        ]

    # Create a new deduper object and pass our data model to it.
    linker = dedupe.RecordLink(fields)
    linker.sample(data_1, data_2, 15000)
    
        # If we have training data saved from a previous run of linker,
    # look for it an load it in.
    # __Note:__ if you want to train from scratch, delete the training_file
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file) as tf :
            linker.readTraining(tf)

    # ## Active learning
    # Dedupe will find the next pair of records
    # it is least certain about and ask you to label them as matches
    # or not.
    # use 'y', 'n' and 'u' keys to flag duplicates
    # press 'f' when you are finished
    print('starting active labeling...')

    dedupe.consoleLabel(linker)

    linker.train()

    # When finished, save our training away to disk
    with open(training_file, 'w') as tf :
        linker.writeTraining(tf)

    # Save our weights and predicates to disk.  If the settings file
    # exists, we will skip all the training and learning next time we run
    # this file.
    with open(settings_file, 'wb') as sf :
        linker.writeSettings(sf)


# ## Blocking

# ## Clustering

# Find the threshold that will maximize a weighted average of our
# precision and recall.  When we set the recall weight to 2, we are
# saying we care twice as much about recall as we do precision.
#
# If we had more data, we would not pass in all the blocked data into
# this function but a representative sample.

print('clustering...')
linked_records = linker.match(data_1, data_2, 0)

print('# duplicate sets', len(linked_records))

# ## Writing Results

# Write our original data back out to a CSV with a new column called 
# 'Cluster ID' which indicates which records refer to each other.

cluster_membership = {}
cluster_id = None
for cluster_id, (cluster, score) in enumerate(linked_records):
    for record_id in cluster:
        cluster_membership[record_id] = (cluster_id, score)

if cluster_id :
    unique_id = cluster_id + 1
else :
    unique_id =0
    

with open(output_file, 'w') as f:
    writer = csv.writer(f)
    
    header_unwritten = True

    for fileno, filename in enumerate(('Caldwell_List_2.csv', 'Data_Model_dedupe.csv')) :
        with open(filename) as f_input :
            reader = csv.reader(f_input)

            if header_unwritten :
                heading_row = next(reader)
                heading_row.insert(0, 'source file')
                heading_row.insert(0, 'Link Score')
                heading_row.insert(0, 'Cluster ID')
                writer.writerow(heading_row)
                header_unwritten = False
            else :
                next(reader)

            for row_id, row in enumerate(reader):
                cluster_details = cluster_membership.get(filename + str(row_id))
                if cluster_details is None :
                    cluster_id = unique_id
                    unique_id += 1
                    score = None
                else :
                    cluster_id, score = cluster_details
                row.insert(0, fileno)
                row.insert(0, score)
                row.insert(0, cluster_id)
                writer.writerow(row)

    

In [ ]:
do=pd.read_csv('data_matching_output.csv')

In [ ]:
do

In [ ]:
matched_dm=do[do['source file']==1]
matched_dm.rename(columns={'Cluster ID':'cluster_id',
'Link Score':'Link_Score',
'source file':'source_file'}).to_csv("matched_data_model.csv",header=True,index=False,encoding='utf8')


In [ ]:
mat_dm=pd.read_csv('matched_data_model.csv')
mat_dm.head()

In [ ]:
matched_cl=do[do['source file']==0]
matched_cl.rename(columns={'Cluster ID':'cluster_id',
'Link Score':'Link_Score',
'source file':'source_file'}).to_csv("matched_caldwell.csv",header=True,index=False,encoding='utf8')

In [ ]:
mat_cl=pd.read_csv('matched_caldwell.csv')
mat_cl.head()